In [327]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.callbacks import ModelCheckpoint

In [376]:
def odds_summary(team_df, team, opp, week, wins, pf=0, pa=0):
    
    def win_rate_by_record(team_df, team, wins, week):
        df = team_df[team_df['team'] == team]
        df = df[df['team_wins'] >= wins]
        df = df[df['week'] == week]
        wins_df = df[df['win'] == 1]
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt
    
    def win_rate_by_pf(team_df, team, pf):
        df = team_df[team_df['team'] == team]
        df = df[df['PF'] >= pf]
        wins_df = df[df['win'] == 1]
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

    def win_rate_by_pa(team_df, team, pa):
        df = team_df[team_df['team'] == team]
        df = df[df['PA'] >= pa]
        wins_df = df[df['win'] == 1]
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt
    
    def win_rate_by_opp(team_df, team, opp):
        df = team_df[team_df['team'] == team]
        df = df[df['opp'] == opp]
        wins_df = df[df['win'] == 1]
        if len(df) == 0:
            return f'No data.'
        else:
            win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
            return win_pt
        
    def avg_pf_by_team(team_df, team, opp):
        df = team_df[team_df['team'] == team]
        df = df[df['opp'] == opp]
        pf_avg = df['PF'].mean()
        pf_avg_rounddown = pf_avg - (pf_avg % 1)
        return pf_avg_rounddown
    
    def avg_pa_by_team(team_df, team, opp):
        df = team_df[team_df['team'] == team]
        df = df[df['opp'] == opp]
        pa_avg = df['PA'].mean()
        pa_avg_roundup = pa_avg + (1 - (pa_avg % 1))
        return pa_avg_roundup
    
    def total_avg(team_df, team, wins, week, opp, pa, pf, current_coach=False, current_qb=False):
        
        if current_coach==False & current_qb==False:
        
            df = team_df[team_df['team'] == team]
            percentage_sum = 0
            count = 0

            # win_rate_by_ record
            win_pt1 = win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)
            percentage_sum = percentage_sum + win_pt1
            count = count + 1

            # win_rate_by_ opp
            df = team_df[team_df['team'] == team]
            df = df[df['opp'] == opp]
            wins_df = df[df['win'] == 1]
            if len(df) == 0:
                pass
            else:
                win_pt2 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
                percentage_sum = percentage_sum + win_pt2
                count = count + 1

            # win_rate_by_ current pf
            win_pt3 = win_rate_by_pf(team_df=team_df, team=team, pf=pf)
            percentage_sum = percentage_sum + win_pt3
            count = count + 1

            # win_rate_by_ current pa
            win_pt4 = win_rate_by_pa(team_df=team_df, team=team, pa=pa)
            percentage_sum = percentage_sum + win_pt4
            count = count + 1

            # win_rate_by_ avg pf
            win_pt5 = win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt5
            count = count + 1

            # win_rate_by_ avg pa
            win_pt6 = win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt6
            count = count + 1

            total_avg = float("{:.2f}".format(percentage_sum/count))
            return total_avg
        
        elif current_coach==True & current_qb==False:
                        
            df = team_df[team_df['team'] == team]
            
            df = df[df['year'] == df['year'].argmax()]
            df = df[df['week'] == df['week'].argmax()]
            current_coach = df['coach']
            
            df = team_df[team_df['team'] == team]
            df = df[df['coach'] == current_coach]
            percentage_sum = 0
            count = 0

            # win_rate_by_ record
            win_pt1 = win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)
            percentage_sum = percentage_sum + win_pt1
            count = count + 1

            # win_rate_by_ opp
            df = team_df[team_df['team'] == team]
            df = df[df['coach'] == current_coach]
            df = df[df['opp'] == opp]
            wins_df = df[df['win'] == 1]
            if len(df) == 0:
                pass
            else:
                win_pt2 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
                percentage_sum = percentage_sum + win_pt2
                count = count + 1

            # win_rate_by_ current pf
            win_pt3 = win_rate_by_pf(team_df=team_df, team=team, pf=pf)
            percentage_sum = percentage_sum + win_pt3
            count = count + 1

            # win_rate_by_ current pa
            win_pt4 = win_rate_by_pa(team_df=team_df, team=team, pa=pa)
            percentage_sum = percentage_sum + win_pt4
            count = count + 1

            # win_rate_by_ avg pf
            win_pt5 = win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt5
            count = count + 1

            # win_rate_by_ avg pa
            win_pt6 = win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt6
            count = count + 1

            total_avg = float("{:.2f}".format(percentage_sum/count))
            return total_avg
        
        elif current_coach==False & current_qb==True:
                        
            df = team_df[team_df['team'] == team]
            
            df = df[df['year'] == df['year'].argmax()]
            df = df[df['week'] == df['week'].argmax()]
            current_qb = df['qb']
            
            df = team_df[team_df['team'] == team]
            df = df[df['qb'] == current_qb]
            percentage_sum = 0
            count = 0

            # win_rate_by_ record
            win_pt1 = win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)
            percentage_sum = percentage_sum + win_pt1
            count = count + 1

            # win_rate_by_ opp
            df = team_df[team_df['team'] == team]
            df = df[df['coach'] == current_coach]
            df = df[df['opp'] == opp]
            wins_df = df[df['win'] == 1]
            if len(df) == 0:
                pass
            else:
                win_pt2 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
                percentage_sum = percentage_sum + win_pt2
                count = count + 1

            # win_rate_by_ current pf
            win_pt3 = win_rate_by_pf(team_df=team_df, team=team, pf=pf)
            percentage_sum = percentage_sum + win_pt3
            count = count + 1

            # win_rate_by_ current pa
            win_pt4 = win_rate_by_pa(team_df=team_df, team=team, pa=pa)
            percentage_sum = percentage_sum + win_pt4
            count = count + 1

            # win_rate_by_ avg pf
            win_pt5 = win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt5
            count = count + 1

            # win_rate_by_ avg pa
            win_pt6 = win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt6
            count = count + 1

            total_avg = float("{:.2f}".format(percentage_sum/count))
            return total_avg
        
        elif current_coach==True & current_qb==True:
            
            df = team_df[team_df['team'] == team]
            
            df = df[df['year'] == df['year'].argmax()]
            df = df[df['week'] == df['week'].argmax()]
            current_coach = df['coach']
            current_qb = df['qb']
            
            df = team_df[team_df['team'] == team]
            df = df[df['coach'] == current_coach]
            df = df[df['qb'] == current_qb]
            percentage_sum = 0
            count = 0

            # win_rate_by_ record
            win_pt1 = win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)
            percentage_sum = percentage_sum + win_pt1
            count = count + 1

            # win_rate_by_ opp
            df = team_df[team_df['team'] == team]
            df = df[df['coach'] == current_coach]
            df = df[df['opp'] == opp]
            wins_df = df[df['win'] == 1]
            if len(df) == 0:
                pass
            else:
                win_pt2 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
                percentage_sum = percentage_sum + win_pt2
                count = count + 1

            # win_rate_by_ current pf
            win_pt3 = win_rate_by_pf(team_df=team_df, team=team, pf=pf)
            percentage_sum = percentage_sum + win_pt3
            count = count + 1

            # win_rate_by_ current pa
            win_pt4 = win_rate_by_pa(team_df=team_df, team=team, pa=pa)
            percentage_sum = percentage_sum + win_pt4
            count = count + 1

            # win_rate_by_ avg pf
            win_pt5 = win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt5
            count = count + 1

            # win_rate_by_ avg pa
            win_pt6 = win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))
            percentage_sum = percentage_sum + win_pt6
            count = count + 1

            total_avg = float("{:.2f}".format(percentage_sum/count))
            return total_avg
        
        else:
            pass
    
    print(f'Avg Win Rating - Overall: {total_avg(team_df=team_df, team=team, wins=wins, week=week, opp=opp, pa=pa, pf=pf)}%')
    print(f'Avg Win Rating - Current Coach: {total_avg(team_df=team_df, team=team, wins=wins, week=week, opp=opp, pa=pa, pf=pf, current_coach=True)}%')
    print(f'Avg Win Rating - Current QB: {total_avg(team_df=team_df, team=team, wins=wins, week=week, opp=opp, pa=pa, pf=pf, current_qb=True)}%')
    print(f'Avg Win Rating - Current Coach & QB: {total_avg(team_df=team_df, team=team, wins=wins, week=week, opp=opp, pa=pa, pf=pf, current_coach=True, current_qb=True)}%')
    print(f'-----')
    print(f'Against {opp}, Win Rate: {win_rate_by_opp(team_df=team_df, team=team, opp=opp)}%')
    print(f'Week {week}- Wins by: {wins}, Win Rate: {win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)}%')
    print(f'Min PF: {pf}, Win Rate: {win_rate_by_pf(team_df=team_df, team=team, pf=pf)}%')
    print(f'Min PA: {pa}, Win Rate: {win_rate_by_pa(team_df=team_df, team=team, pa=pa)}%')
    print(f'Avg PF: {avg_pf_by_team(team_df=team_df, team=team, opp=opp)}, Win Rate: {win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))}%')
    print(f'Avg PF: {avg_pa_by_team(team_df=team_df, team=team, opp=opp)}, Win Rate: {win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))}%')

In [359]:
teams_list = ['Cardinals', 'Falcons', 'Ravens', 'Bills', 'Panthers', 'Bears', 'Bengals', 'Browns', 
              'Cowboys', 'Broncos', 'Lions', 'Packers', 'Texans', 'Colts', 'Jaguars', 'Chiefs', 
              'Chargers', 'Rams', 'Dolphins', 'Vikings', 'Patriots', 'Saints', 'Giants', 'Jets', 
              'Raiders', 'Eagles', 'Steelers', '49ers', 'Seahawks', 'Buccaneers', 'Titans', 'Redskins']

week_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [360]:
season = 2019
team = '49ers'
coach = 'Shanahan, Kyle'
opp_list = ['Buccaneers', 
            'Bengals', 
            'Steelers', 
            'BYE', 
            'Browns', 
            'Rams',
            'Redskins', 
            'Panthers', 
            'Cardinals', 
            'Seahawks', 
            'Cardinals',
            'Packers', 
            'Ravens', 
            'Saints', 
            'Falcons', 
            'Rams', 
            'Seahawks']
home_list = [0,0,1,'BYE',1,0,0,1,0,1,1,1,0,0,1,1,0]
win_list = [1, 1, 1, 'BYE', 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1]
pf_list = [21,41,24,'BYE',31,20,9,51,28,24,36,37,17,48,22,34,26]
pa_list = [17,17,20,'BYE',3,7,0,13,25,27,26,8,20,46,29,31,21]
team_wins = [0,1,2,'BYE',3,4,5,6,7,8,8,9,10,10,11,11,12]
week_after_bye = [0,0,0,'BYE',1,0,0,0,0,0,0,0,0,0,0,0,0]

sf_df2019 = pd.DataFrame({'season': season, 
                          'week': week_list, 
                          'team': team,
                          'coach': coach,
                          'opp': opp_list,
                          'home': home_list,
                          'win': win_list,
                          'PF': pf_list,
                          'PA': pa_list, 
                          'team_wins': team_wins})

In [361]:
season = 2018
team = '49ers'
coach = 'Shanahan, Kyle'
opp_list = ['Vikings', 
            'Lions', 
            'Chiefs', 
            'Chargers', 
            'Cardinals', 
            'Packers',
            'Rams', 
            'Cardinals', 
            'Raiders', 
            'Giants', 
            'BYE', 
            'Buccaneers', 
            'Seahawks', 
            'Broncos', 
            'Seahawks', 
            'Bears', 
            'Rams']
home_list = [0,1,0,0,1,0,1,0,1,1,'BYE',0,0,1,1,1,0]
win_list = [0,1,0,0,0,0,0,0,1,0,'BYE',0,0,1,1,0,0]
pf_list = [16,30,27,27,18,30,10,15,34,23,'BYE',9,16,20,26,9,32]
pa_list = [24,27,38,29,28,33,39,18,3,27,'BYE',27,43,14,23,14,48]
team_wins = [0,0,1,1,1,1,1,1,1,2,'BYE',2,2,2,3,4,4]
week_after_bye = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]

sf_df2018 = pd.DataFrame({'season': season, 
                          'week': week_list, 
                          'team': team,
                          'coach': coach,
                          'opp': opp_list,
                          'home': home_list,
                          'win': win_list,
                          'PF': pf_list,
                          'PA': pa_list, 
                          'team_wins': team_wins})

In [362]:
season = 2017
team = '49ers'
coach = 'Shanahan, Kyle'
opp_list = ['Panthers', 
            'Seahawks',
            'Rams', 
            'Cardinals',
            'Colts', 
            'Redskins', 
            'Cowboys', 
            'Eagles', 
            'Cardinals', 
            'Giants', 
            'BYE', 
            'Seahawks', 
            'Bears', 
            'Texans', 
            'Titans', 
            'Jaguars', 
            'Rams']
home_list = [1,0,1,0,0,0,1,0,1,1,'BYE',1,0,0,1,1,0]
win_list = [0,0,0,0,0,0,0,0,0,1,'BYE',0,1,1,1,1,1]
pf_list = [3,9,39,15,23,24,10,10,10,31,'BYE',13,15,26,25,44,34]
pa_list = [23,12,41,18,26,26,40,33,20,21,'BYE',24,14,16,23,33,13]
team_wins = [0,0,0,0,0,0,0,0,0,0,'BYE',1,1,2,3,4,5]
week_after_bye = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]

sf_df2017 = pd.DataFrame({'season': season, 
                          'week': week_list, 
                          'team': team,
                          'coach': coach,
                          'opp': opp_list,
                          'home': home_list,
                          'win': win_list,
                          'PF': pf_list,
                          'PA': pa_list, 
                          'team_wins': team_wins})

In [363]:
season = 2016
team = '49ers'
coach = 'Kelly, Chip'
opp_list = ['Rams', 
            'Panthers',
            'Seahawks', 
            'Cowboys',
            'Cardinals', 
            'Bills', 
            'Buccaneers', 
            'BYE', 
            'Saints', 
            'Cardinals', 
            'Patriots', 
            'Dolphins', 
            'Bears', 
            'Jets', 
            'Falcons', 
            'Rams', 
            'Seahawks']
home_list = [1,0,0,1,1,0,1,'BYE',1,0,1,0,0,1,0,0,1]
win_list = [1,0,0,0,0,0,0,'BYE',0,0,0,0,0,0,0,1,0]
pf_list = [28,27,18,17,21,16,17,'BYE',23,20,17,24,6,17,13,22,23]
pa_list = [0,46,37,24,33,45,34,'BYE',41,23,30,31,26,23,41,21,25]
team_wins = [0,1,1,1,1,1,1,'BYE',1,1,1,1,1,1,1,1,2]
week_after_bye = [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]

sf_df2016 = pd.DataFrame({'season': season, 
                          'week': week_list, 
                          'team': team,
                          'coach': coach,
                          'opp': opp_list,
                          'home': home_list,
                          'win': win_list,
                          'PF': pf_list,
                          'PA': pa_list, 
                          'team_wins': team_wins})

In [ ]:
season = 2015
team = '49ers'
coach = 'Tomsula, Jim'
#opp_list = ['Rams', 
            'Panthers',
            'Seahawks', 
            'Cowboys',
            'Cardinals', 
            'Bills', 
            'Buccaneers', 
            'BYE', 
            'Saints', 
            'Cardinals', 
            'Patriots', 
            'Dolphins', 
            'Bears', 
            'Jets', 
            'Falcons', 
            'Rams', 
            'Seahawks']
#home_list = [1,0,0,1,1,0,1,'BYE',1,0,1,0,0,1,0,0,1]
#win_list = [1,0,0,0,0,0,0,'BYE',0,0,0,0,0,0,0,1,0]
#pf_list = [28,27,18,17,21,16,17,'BYE',23,20,17,24,6,17,13,22,23]
#pa_list = [0,46,37,24,33,45,34,'BYE',41,23,30,31,26,23,41,21,25]
#team_wins = [0,1,1,1,1,1,1,'BYE',1,1,1,1,1,1,1,1,2]
#week_after_bye = [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]

sf_df2015 = pd.DataFrame({'season': season, 
                          'week': week_list, 
                          'team': team,
                          'coach': coach, 
                          'opp': opp_list,
                          'home': home_list,
                          'win': win_list,
                          'PF': pf_list,
                          'PA': pa_list, 
                          'team_wins': team_wins})

In [364]:
sf_df = sf_df2019.append(sf_df2018, ignore_index=True)
sf_df = sf_df.append(sf_df2017, ignore_index=True)
sf_df = sf_df.append(sf_df2016, ignore_index=True)
#sf_df = sf_df.append(sf_df2015, ignore_index=True)
#sf_df = sf_df.append(sf_df2014, ignore_index=True)
#sf_df = sf_df.append(sf_df2013, ignore_index=True)
#sf_df = sf_df.append(sf_df2012, ignore_index=True)
#sf_df = sf_df.append(sf_df2011, ignore_index=True)
#sf_df = sf_df.append(sf_df2010, ignore_index=True)
sf_df = sf_df[sf_df['opp'] != 'BYE']

In [368]:
odds_summary(team_df=sf_df, team='49ers', opp='Giants', week=3, wins=1, pf=0, pa=0)

Avg Win Rating: 41.6%
-----
Against Giants, Win Rate: 50.0%
Week 3- Wins by: 1, Win Rate: 33.33%
Min PF: 0, Win Rate: 39.06%
Min PA: 0, Win Rate: 39.06%
Avg PF: 27.0, Win Rate: 70.0%
Avg PF: 25.0, Win Rate: 18.18%
